In [83]:
using Plots, LaTeXStrings , Distributions, StatsPlots , Random, PlotlyJS

In [84]:
plotlyjs()                              # Deciding to make the plot interactive or not
#gr()


Plots.PlotlyJSBackend()

In [85]:
N=100000

function Metropolis(func, distance, n)
    Position = 0.0
    count = 0
    PositionArray = zeros(n)
    for i in 1:n 
        PositionArray[i] = Position                                                     # Saving the X poistions of ball moving throgh the domain.
        ProbableDestination = Position + distance * rand(Uniform(-1,1))
        TransitionProbability = func(ProbableDestination)/func(Position)                # Cheking if the ball is going to move to a new position
        if rand() <= TransitionProbability
            Position = ProbableDestination
            count = count + 1
        end
    end
    AcceptenceRate = count/n                                                            # Calculating acceptence Rate
    return PositionArray, AcceptenceRate
end

function normal(x,sigma = 1 ,miu = 0)                                                   # Normal distribution function
    f = (1/(sqrt(2*pi))*sigma)*exp(-(0.5)*((x-miu)/sigma)^2)
    return f
end




normal (generic function with 3 methods)

Plotting

In [86]:
#Plotting
data , AccRate = Metropolis(normal, 10, N)
stephist(data,normalized=true, fillcolor = "darkorchid4" , fill = true , alpha = 0.7 , label = "Metropolice Algorithm")
plot!(normal , color = "darkblue" , lw = 2, label = "Normal distribution")  
plot!(xlabel = L"x" , ylabel = "density", title = "Density of Positions with N=100000" )  


In [88]:
AccRateArray = zeros(2000)
counter = 1
for i in (0.01):0.01:(20)
    AccRateArray[counter] = Metropolis(normal, i, N)[2]     #Loop for saving Accept Rates in an array
    counter +=1 

end


In [6]:
Plots.plot( [(0.01):0.01:(20)], AccRateArray, showlegend = false)
Plots.plot!(xlabel = "Step Length", ylabel = "Acceptance Rate", title = "Acceptance Rate ")

Auto-Corelation

In [23]:
function AutoCorrelation(array,n)
    AutoCorr = zeros(n)
    @simd for j in  1:n
        AutoCorr[j] = (sum((array[1:(n-j)] .- mean(array)) .* (array[(j+1):(n)] .- mean(array))))/ (N*var(array))         # Using Auto-Correlation Formula mentioned in report
    end
    return AutoCorr
end

AutoCorrelationArray = AutoCorrelation(data,N)
;

In [56]:
Plots.plot(AutoCorrelationArray[1:100], label = "AutoCorrealtion for step Length = 10")
plot!(xlabel = "lag", ylabel = "AutoCorrelation[lag]", title = "Auto-Correlation")

In [26]:
AcceptanceRates = [16.13,8.05,5.21,3.87,2.93,2.16,1.55,1.01,0.45]
MetroResult = []
for Acce in AcceptanceRates
    push!(MetroResult , Metropolis(normal, Acce, N)[1])
end
CorrelationList = []
for i in 1:9
    push!(CorrelationList, AutoCorrelation(MetroResult[i], N))
end
    

In [27]:
CorrelationList

9-element Vector{Any}:
 [0.8665510276774039, 0.7523942927270293, 0.6545612816378107, 0.5694006966761944, 0.4955912691643748, 0.4315108237452287, 0.37571003970451894, 0.3278809941948956, 0.2860813901405972, 0.2504511958501709  …  -1.8558424127194232e-5, 8.069482847022966e-7, 7.521157284057411e-7, 6.446706243477781e-7, 5.37225520289815e-7, 4.2978041623185203e-7, 3.2233531217388904e-7, 2.1489020811592602e-7, 1.0744510405796301e-7, 0.0]
 [0.7367035019365267, 0.5484191458764786, 0.4076606450520838, 0.30612582537347993, 0.23139203237392045, 0.17597444481666566, 0.13600615890307566, 0.10559970903583651, 0.0819606044975999, 0.06254098471550434  …  -5.3026971143319967e-5, -5.072237178584809e-5, -4.841777242837622e-5, -4.6113173070904346e-5, -4.380857371343247e-5, -4.6066576444367204e-5, -4.340824602792485e-5, -4.07943193288418e-5, 3.7522729720390036e-7, 0.0]
 [0.6135819437069189, 0.3815959488132095, 0.2408806280083454, 0.15410500732274418, 0.10204426389447639, 0.06994460833134399, 0.04897021510

In [69]:
Plots.plot()
for i in 1:9
    Plots.plot!(CorrelationList[i][1:100], label = "AccRate: $(round(0.1*i,digits = 2))")

end
Plots.plot!(xlabel = "lag" , ylabel ="AutoCorrelation", title = "Auto-Correlation for different Step Lengths")


In [82]:
correlationLogList = []
correlationlengthList = zeros(9)
for i in 1:9
    push!(correlationLogList , log.(abs.(CorrelationList[i][1:8])))
    para = collect(1:8)  \ correlationLogList[i]
    correlationlengthList[i] = -1 / para

end

Plots.plot(correlationlengthList, label = "Correlation Length")
Plots.plot!(xlabel = "Acceptance Rate" , ylabel = "correlation length" , titel = "Corelation Length" )
    